#### **Setup for fine-tuning and deploying Meta/llama2 model for Salesforce customer service dataset**

In this notebook, I use a qunatized version of llama2-base and PEFT/LoRA to reduce update size. The model is then fine-tuned on a Lightning.io T4 GPU (16GB) RAM where it trained well.

In [1]:
!pip install torch==2.0.1 
!pip install transformers==4.32.1
!pip install datasets==2.14.4
!pip install peft==0.5.0
!pip install bitsandbytes==0.41.1
!pip install trl==0.7.1

In [1]:
import json
import re
from pprint import pprint


import pandas as pd
import torch
from datasets import Dataset, load_dataset

from huggingface_hub import notebook_login
from peft import LoraConfig, PeftModel

from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    BitsAndBytesConfig,
    TrainingArguments)

from trl import SFTTrainer

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

MODEL_NAME = "meta-llama/Meta-Llama-3-8B-Instruct"


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


#### **1. Download and prepare Salesforce customer service dataset**

In [3]:
import os
from src import text_utils
os.environ["CURL_CA_BUNDLE"] = ""
dataset = load_dataset("Salesforce/dialogstudio", "TweetSumm")

dataset["train"] = text_utils.process_dataset(dataset["train"])
dataset["validation"] = text_utils.process_dataset(dataset["validation"])

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/datasets/load.py:1486: FutureWarning: The repository for Salesforce/dialogstudio contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Salesforce/dialogstudio
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [4]:
dataset["train"]

Dataset({
    features: ['conversation', 'summary', 'text'],
    num_rows: 879
})

In [17]:
notebook_login()

### **2. Download model weights and Fine-tune Llama-3**:

In [6]:
def create_model_and_tokenizer():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )
 
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        #new_model,
        use_safetensors=True,
        quantization_config=bnb_config,
        trust_remote_code=True,
        device_map="auto",
    )
 
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
 
    return model, tokenizer

In [7]:
model, tokenizer = create_model_and_tokenizer()
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
model.config.quantization_config.to_dict()

{'quant_method': <QuantizationMethod.BITS_AND_BYTES: 'bitsandbytes'>,
 'load_in_8bit': False,
 'load_in_4bit': True,
 'llm_int8_threshold': 6.0,
 'llm_int8_skip_modules': None,
 'llm_int8_enable_fp32_cpu_offload': False,
 'llm_int8_has_fp16_weight': False,
 'bnb_4bit_quant_type': 'nf4',
 'bnb_4bit_use_double_quant': False,
 'bnb_4bit_compute_dtype': 'float16'}

In [9]:
for example in dataset['validation']:
    if 'text' not in example.keys():
        print(example)

In [10]:
lora_r = 16
lora_alpha = 64
lora_dropout = 0.1
lora_target_modules = [
    "q_proj",
    "up_proj",
    "o_proj",
    "k_proj",
    "down_proj",
    "gate_proj",
    "v_proj",
]
 
 
peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=lora_target_modules,
    bias="none",
    task_type="CAUSAL_LM",
)

In [12]:
OUTPUT_DIR = "experiments"
 
%load_ext tensorboard
%tensorboard --logdir experiments/runs

/system/conda/miniconda3/envs/cloudspace/lib/python3.10/site-packages/google/colab/data_table.py:30: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  from IPython.utils import traitlets as _traitlets


AttributeError: module 'IPython.utils.traitlets' has no attribute 'Unicode'

In [13]:
training_arguments = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=1e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=2,
    evaluation_strategy="steps",
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy="epoch",
    group_by_length=True,
    output_dir=OUTPUT_DIR,
    report_to="tensorboard",
    save_safetensors=True,
    lr_scheduler_type="cosine",
    seed=42,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=4096,
    tokenizer=tokenizer,
    args=training_arguments,
)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
CUDA extension not installed.
CUDA extension not installed.


In [14]:
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
88,1.780000,2.142741
176,2.134700,2.094123
264,1.642400,2.115218
352,1.657600,2.088345
440,1.550600,2.091321


TrainOutput(global_step=440, training_loss=1.9944630433212627, metrics={'train_runtime': 2038.2086, 'train_samples_per_second': 0.863, 'train_steps_per_second': 0.216, 'total_flos': 1.3490074611769344e+16, 'train_loss': 1.9944630433212627, 'epoch': 2.0})

#### **Save fine-tuned model for later use**

In [18]:
from peft import AutoPeftModelForCausalLM
 
trained_model = AutoPeftModelForCausalLM.from_pretrained(
    OUTPUT_DIR,
    low_cpu_mem_usage=True,
)
 
merged_model = model.merge_and_unload()
merged_model.save_pretrained("merged_model_ll3", safe_serialization=True)
tokenizer.save_pretrained("merged_model_ll3")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

AttributeError: 'LlamaForCausalLM' object has no attribute 'merge_and_unload'

### **Evaluate model after fine-tuning**